# Desenvolvimento de API com Flask parte 2

- Definições: iniciar a configuração do Flask com banco de dados e JWT
- Contexto: vamos apresentar a classe Config e criar nosso banco (SQLite)
- Dinâmica: ao final da aula, teremos um projeto Flask completo
- Estruturação: foco no setup inicial (ORM + Models) antes das rotas

## Parte 1

**Objetivos da parte 1**

- Passo 1: mostrar a classe Config com informações de SWAGGER, SECRET_KEY e DB
- Passo 2: instanciar Flask e aplicar configurações (app.config.from_object(Config))
- Passo 3: integrar SQLAlchemy ao projeto, explicando vantagens do ORM
- Passo 4: criar modelos User e Recipe com atributos básicos
- Passo 5: testar a criação de tabelas usando db.create_all()

### API de receitas gourmet

**A classe config**

Mas muitas vezes será necessário configurar o projeto previamente!

Por exemplo, você pode querer, antes de começar o projeto, configurar:

- Uma chave secreta para ele
- Mudar o projeto para o modo de teste
- Entre outros

Para isso, em Flask temos a opção de alterar as configurações via:

```py
app.config
```

Documentação: https://flask.palletsprojects.com/en/stable/config/

```py
from flask import Flask


#configurações da aplicação
class Config:
    SECRET_KEY = 'sua_chave_secreta'
    CACHE_TYPE = 'simple'
    SWAGGER = {
        'title': 'Catálogo de receitas gourmet',
        'uiversion': 3
    }
    SQLALCHEMY_DATABASE_URI = 'sqlite:///recipes.db'
    SQLALCHEMY_TRACK_MODIFICATIONS = False
    JWT_SECRET_KEY = 'sua_chave_secreta'

app = Flask(__name__)
app.config.from_object(Config)

print(app.config['SECRET_KEY'])
print(app.config['SQLALCHEMY_DATABASE_URI'])
print(app.config['SWAGGER'])
print(app.config['CACHE_TYPE'])
```

Fizemos então a criação de uma classe "Config", que possui as configurações dos projetos. Colocamos ela então no app.config. Outra opção interessante seria criar um arquivo .py auxiliar com as configurações e as importar. Por exemplo:

```py
from flask import Flask
from config import Config


app = Flask(__name__)

app.config.from_object(Config)

print(app.config['SECRET_KEY'])
print(app.config['SQLALCHEMY_DATABASE_URI'])
print(app.config['SWAGGER'])
print(app.config['CACHE_TYPE'])
```

Mas o que são essas configurações? Como sei as possíveis? Há diversas, como as nativas: DEBUG, TESTING, SECRET_KEY... E outras de extensões também: SQLALCHEMY_DATABASE_URI, JWT_SECRET_KEY

Leia a documentação para ver muitas outras opções:

https://flask.palletsprojects.com/en/stable/config/
https://flask-sqlalchemy.readthedocs.io/en/stable/config/

Vamos utilizar por enquanto:

- SECRET_KEY e JWT_SECRET_KEY para segurança
- CACHE_TYPE = 'simple' habilita caching básico
- SWAGGER config para título e versão da doc interativa
- SQLALCHEMY_DATABASE_URI define qual banco (ex.: sqlite:///recipes.db)
- SQLALCHEMY_TRACK_MODIFICATIONS = False para evitar warnings

### Banco de dados e SQLAlchemy

O que é um banco de dados?

- Sistema que armazena e organiza dados de forma estruturada
- Permite acesso, gerenciamento e manipulação de dados
- Exemplos: SQLite, MySQL, PostgreSQL, Oracle

O que é ORM?

- Mapeamento entre tabelas do banco e objetos/classe no código
- Facilita o uso de banco de dados sem comandos SQL diretos
- Garante consistência e facilita a manutenção do código
- SQLAlchemy é um ORM (Object-Relational Mapper)

### Como criar modelos?

- Cada modelo representa uma tabela no banco
- Definido como uma classe que herda de db.Model
- Colunas definidas como atributos de classe

```py
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    password = db.Column(db.String(120), nullable=False)
```

Instalação e implementação

```py
!pip install flask-sqlalchemy
```

```py
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)

app.config.from_object(Config)

db = SQLAlchemy(app)

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    password = db.Column(db.String(120), nullable=False)

class Recipe(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(120), nullable=False)
    ingredients = db.Column(db.Text, nullable=False)
    time_minutes = db.Column(db.Integer, nullable=False)

if __name__ == '__main__': #rodando localmente
    with app.app_context():
        db.create_all()
        print('Banco de dados criado!')
```

### Autenticação

Como vimos em aulas anteriores, temos diversas opções de autenticação, como:

- Auth Básica
- JWT

**Autenticação JWT (JSON WEB TOKEN)**

- Padrão para autenticação e troca segura de informações em formato compacto
- Se divide em três partes (Header, Payload e Signature) que garantem a estrutura e a validade dos dados
- Após o login, o servidor gera um token que o cliente envia em cada requisição para validar seu acesso
- O servidor não precisa manter sessões, pois a verificação do token ocorre no próprio conteúdo assinado
- É amplamente utilizado em APIs REST, sistemas de login, Single Sign-On (SSO) e microsserviços

**Autenticação JWT na prática**

Instalando biblioteca biblioteca necessária:

```py
!pip install Flask-JWT-Extended
```

Importar o necessário:

```py
from flask_jwt_extended import (
    JWTManager, create_access_token,
    jwt_required, get_jwt_identity
)
```

E instanciar o JWTManager:

```py
db = SQLAlchemy(app)
jwt = JWTManager(app)
```

**Fluxo de login com JWT**

- Usuário chama /login passando username e password
- Se credenciais válidas, geramos token com create_access_token(identity=user.id);
- Resposta: {"access_token": "<jwt_token>"}
- Em chamadas subsequentes, usuário envia Authorization: Bearer <token>
- Rotas protegidas exigem @jwt_required() no decorator

Criando rotas de registro de usuário e login

```py
@app.route('/register', methods=['POST'])
def register_user():
    '''
    Registra um novo usuário.
    ---
    parameters:
        - in: body
            name: body
            required: true
            schema:
                type: object
            properties:
                username:
                    type: string
                password:
                    type: string
    responses:
        201:
            description: Usuário criado com sucesso
        400:
            description: Usuário já existe
    '''
    data = request.get_json()
    if User.query.filter_by(username=data['username']).first():
        return jsonify({'error': 'User already exists'}), 400
    new_user = User(username=data['username'], password=data['password'])
    db.session.add(new_user)
    db.session.commit()
    return jsonify({'msg': 'User created'})

@app.route('/login', methods=['POST'])
def login():
    '''
    Faz login do usuário e retorno um JWT.
    ---
    parameters:
        - in: body
            name: body
            required: true
            schema:
                type: object
            properties:
                username:
                    type: string
                password:
                    type: string
    responses:
        200:
            description: login bem sucedido, retorna JWT
        401:
            description: Credenciais inválidas
    '''
    data = request.get_json()
    user = User.query.filter_by(username=data['username']).first()
    if user and user.password == data['password']:
        #converte o ID para string
        token = create_access_token(identity=str(user.id))
        return jsonify({'access_token': token}), 200
    return jsonify({'error': 'Invalid credentials'})
```

A ideia é utilizar o token criando uma rota simples:

```py
@pp.route('/protected', methods=['GET'])
@jwt_required()
def protected()
    current_user_id = get_jwt_identity() #retorna o identity usado na criação do token
    return jsonify({'msg': f'Usuário com ID {current_user_id} acessou a rota protegida'})
```

**Revisão**

- /register -> cria usuário (sem token)
- /login -> autentica e retorna token
- No próximo vídeo: /recipes e /scrape exigirão token
- /apidocs para documentação
- Aplicação ganha forma de API robusta

### API completa

- Criar rotas /recipes (POST e GET) com filtros e caching
- Rotas /recipes/<int:recipe_id> (PUT e DELETE)
- Exigir token JWT em todas as rotas de CRUD
- Mostrar scraping (/scrape/title, /scrape/content) autenticado
- Concluir aplicação e discutir próximos passos

Vamos começar criando rotas de receita com os seguintes propósitos:

- /recipes (POST) -> cria receita, token obrigatório
- /recipes (GET) -> lista receitas (com cache) + filtros
- /recipes/<id> (PUT) -> atualiza receita existente
- /recipes/<id> (DELETE) -> remove receita específica
- Todos usam @jwt_required() para validação do token

Rota POST de receitas:

```py
@app.route('/recipes', methods=['POST'])
@jwt_required()
def create_recipe():
    '''
    Cria uma nova receita.
    ---
    security:
        - BearerAuth: []
    parameters:
        - in: body
          name: body
          schema:
            type: object
            required: true
            properties:
                title:
                    type: string
                ingredients
                    type: string
                time_minutes
                    type: integer
    responses:
        201:
            description: Receita criada com sucesso
        401:
            description: Token não fornecido ou inválido
    '''
    data = request.get_json()
    new_recipe = Recipe(
        title=data['title'],
        ingredients=data['ingredients'],
        time_minutes=data['time_minutes']
    )
    db.session.add(new_recipe)
    db.session.commit()
    return jsonify({'msg': 'Recipe created'})
```

Rota GET de receitas:

```py
@app.route('/recipes', methods=['GET'])
def get_recipes():
    '''
    Lista receitas com filtros opcionais.
    ---
    parameters:
        - in: query
          name: ingredient
          type: string
          required: false
          description: Filtra por ingrediente
        - in: query
          name: max_time
          type: integer
          required: false
          description: Tempo máximo de preparo
    responses:
        200:
            description: Lista de receitas filtradas
            schema:
                type: array
                items:
                    type: object
                    properties:
                        id:
                            type: integer
                        title:
                            type: string
                        time_minutes:
                            type: integer
    '''
    ingredient = request.args.get('ingredients')
    max_time = request.args.get('max_time', type=int)

    query = Recipe.query
    if ingredient:
        query = query.filter(Recipe.ingredients.ilike(f'%{ingredient}%'))
    if max_time is not None:
        query = query.filter(Recipe.time_minutes <= max_time)

    recipes = query.all()
    return jsonify([
        {
            'id': r.id,
            'title': r.title,
            'ingredients': r.ingredients,
            'time_minutes': r.time_minutes
        }
        for r in recipes
    ])
```

Rota PUT de receitas:

```py
@app.route('/recipes/<int:recipe_id>', methods=['PUT'])
@jwt_required()
def update_recipe(recipe_id):
    '''
    Atualiza uma receita existente.
    ---
    security:
        - BearerAuth: []
    parameters:
        - in: path
            name: recipe_id
            required: true
            type: integer
        - in: body
            name: body
            schema:
                type: object
                properties:
                    title:
                        type: string
                    ingredients:
                        type: string
                    time_minutes:
                        type: integer
    responses:
        200:
            description: Receita atualizada
        404:
            description: Receita não encontrada
        401:
            description: Token não fornecido ou inválido
    '''
    data = request.get_json()
    recipe = Recipe.query.get_or_404(recipe_id)
    if 'title' in data:
        recipe.title = data['title']
    if 'ingredients' in data:
        recipe.ingredients = data['ingredients']
    if 'time_minutes' in data:
        recipe.time_minutes = data['time_minutes']
    db.session.commit()
    return jsonify({"msg": "Recipe updated"}), 200
```

Rota DELETE de receitas:

```py
@app.route('/recipes/<int:recipe_id>', methods=['DELETE'])
@jwt_required()
def delete_recipe(recipe_id):
    '''
    Remove uma receita existente.
    ---
    security:
        - BearerAuth: []
    parameters:
        - in: path
            name: recipe_id
            required: true
            type: integer
    responses:
        200:
            description: Receita removida com sucesso
        404:
            description: Receita não encontrada
        401:
            description: Token não fornecido ou inválido
    '''
    # Lógica de exclusão correta (Busca, Deleta e Commita)
    recipe = Recipe.query.get_or_404(recipe_id)
    db.session.delete(recipe)
    db.session.commit()
    return jsonify({"msg": "Recipe deleted"}), 200
```

**Documentação com Swagger**

- Swagger é uma documentação interativa para APIs
- Facilita a visualização e teste de endpoints
- Integração simples com Flask usando bibliotecas como Flasgger
- Permite definir parâmetros e respostas diretamente no código
- Garante padronização e validação dos dados da API